ETL Process

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from queries_sql import songs_table_insert, artists_table_insert, times_table_insert, users_table_insert, songplays_table_insert, songs_select

In [16]:
try: 
    conn = psycopg2.connect("dbname=sparkifydb user=postgres password=admin123")
    print("Connection to the database is successful")
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

Connection to the database is successful


Get file path:

In [3]:
def getFiles(filePath):
    all_files = []
    # root: đường dẫn thư mục hiện tại
    # dirs: danh sách thư mục con
    # files: danh sách file trong thư mục hiện tại
    for root, dirs, files in os.walk(filePath):
        # print(root, dirs, files)
        files = glob.glob(os.path.join(root, '*.json')) 
        for f in files:
            all_files.append(os.path.abspath(f))
        
    return all_files

Process song_data:

In [4]:
song_df = []

for song_path in getFiles('data/song_data'):
    song_data = pd.read_json(song_path, lines=True)
    song_df.append(song_data)

song_dfs = pd.concat(song_df)
song_dfs

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
...,...,...,...,...,...,...,...,...,...,...
0,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
0,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
0,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
0,1,ARWB3G61187FB49404,NaN,NaN,"Hamilton, Ohio",Steve Morse,SODAUVL12A8C13D184,Prognosis,363.85914,2000


1. Extract to songs table (convert into list and insert)

In [ ]:

try:
    for song_path in getFiles('data/song_data'):
        song_data = pd.read_json(song_path, lines=True)
        cur.execute(songs_table_insert, list(song_data[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]))
        conn.commit()
    print("Insert data into song table successfully")
except Exception as e:
    print(e)

2. Extract to artists table (convert into list and insert)

In [30]:
try:
    for artist_path in getFiles('data/song_data'):
        artist_data = pd.read_json(artist_path, lines=True)
        cur.execute(artists_table_insert, list(artist_data[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]))
        conn.commit()
    print("Insert data into artist table successfully")
except Exception as e:
    print(e)

Insert data into artist table successfully


Process log_data:

In [5]:
log_df = []

for log_path in getFiles('data/log_data'):
    log_data = pd.read_json(log_path, lines=True)
    log_df.append(log_data)

logs_df = pd.concat(log_df)
logs_df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,1543603476796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
384,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,1543603678796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
385,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,1543603884796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
386,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,1543603993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


3. Extract to times table
    - Filter records by NextSong action
    - Convert the ts timestamp column to datetime
    - Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
    - Specify labels for these columns and set to column_labels
    - Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

In [ ]:

for log_path in getFiles('data/log_data'):
    og_data = pd.read_json(log_path, lines=True)
    og_data = og_data[og_data['page'] == 'NextSong']
    t = pd.to_datetime(og_data['ts'])

In [46]:
time_data = [(tt.value, tt.hour, tt.day, tt.week, tt.month, tt.year, tt.weekday()) for tt in t]
column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')

<class 'tuple'>


Take time values from files and convert into dataframe

In [41]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df

,timestamp,hour,day,week,month,year,weekday
0,1543537327796,0,1,1,1,1970,3
1,1543540121796,0,1,1,1,1970,3
2,1543540368796,0,1,1,1,1970,3
3,1543540625796,0,1,1,1,1970,3
4,1543540856796,0,1,1,1,1970,3
...,...,...,...,...,...,...,...
325,1543603205796,0,1,1,1,1970,3
326,1543603476796,0,1,1,1,1970,3
327,1543603678796,0,1,1,1,1970,3
328,1543603884796,0,1,1,1,1970,3


Insert Records into times table

In [ ]:
try:
    for i, row in time_df.iterrows():
        cur.execute(times_table_insert, list(row))
        conn.commit()
    print("Insert data into time table successfully")
except Exception as e:
    print(e)

4. Extract to users table

In [ ]:
try: 
    user_df = logs_df[['userId', 'firstName', 'lastName', 'gender', 'level']]
    for i, row in user_df.iterrows():
        print(list(row))
        cur.execute(users_table_insert, list(row))
        conn.commit()
    print("Insert data into user table successfully")
except Exception as e:
    print(e)

5. Extract songplays table

In [ ]:
try:
    
    for index, row in logs_df.iterrows():
        # Tham so truyen vao cau lenh sql
        cur.execute(songs_select, (row.song, row.artist, row.length))
        results = cur.fetchone()    
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = (index, row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'],
                     row['location'], row['userAgent'])
        print(song_data)
        cur.execute(songplays_table_insert, songplay_data)
        conn.commit()
except Exception as e:
    print(e)

In [15]:
conn.close()